# Coastal Flood Risk Visualization

This visualization uses a gpu to plot households and tanks on a dashboard, with range slider features to show households that have a certain risk index range. Also we have included the elderly multiselect, and household/tank distance slider.

### Import Statements

In [ ]:
import geopandas as gpd

import cuxfilter
from cuxfilter.layouts import feature_and_five_edge, double_feature
import cudf
import numpy as np

import holoviews as hv
import pandas as pd

from pyproj import Proj, Transformer



RuntimeError: Failed to dlopen libcuda.so

### Importing Hurricane Risk and Distance Data

This is a preprocessed parquet file that includes the latitude and longitude of households and tanks that are already transformed into the 3857 projections. This file also contains the ```is_elderly``` variable that indicates if the point at a particular lat/lon is a household with elderly, a household without elderly, or a tank. ```distance_mi``` is the distance in miles between tanks and houses. 

In [ ]:
df = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/coast_flood_risks_final.parquet')
df

### Transforming the dataframe type

Here, we are transforming the pandas dataframe into a cudf dataframe and then into a cuxfilter dataframe so that we can plot the points using the cuxfilter library.

In [ ]:
cdf = cudf.DataFrame.from_pandas(df) 

In [ ]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf) 

### Creating multiselect label map and defining map color palette

In the label map for the ```is_elderly``` multiselect, we are defining the labels for the values that equal 0, 1, and 2 respectively.

In [ ]:
label_elderly = {0: 'Tank', 1: 'Elderly', 
             2: 'No Elderly'}

from bokeh.palettes import Spectral11
colors = list(Spectral11)
colors.reverse()


### Defining charts

Below, we are defining the four charts that will be plotted on our dashboard. In each chart, you can define the type of cuxfilter chart (```scatter```, ```multiselect```, ```range_slider```, ```drop_down```, etc.) and also specify the column that the chart is made on. The label map parameter specifies which label map the feature will pull from. 

In [ ]:
household = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'distance_mi', tile_provider="CartoDark", title = 'Households and distances to nearest tank, with coastal flooding risk slider', legend = True)

elderly = cuxfilter.charts.multi_select('is_elderly', label_map = label_elderly)

coast_flood_risk= cuxfilter.charts.range_slider('coastal_flood_risk')

dist = cuxfilter.charts.range_slider('distance_mi')


### Plotting the charts on the dashboard

The layout and sidebar parameter determines how the charts are oganized on the dashboard. Here we have put three charts as the sidebar, however, you could also place those three charts underneath the main chart by specifying feature and base under the layout feature.

In [ ]:
d = cux_df.dashboard([household], sidebar = [elderly, coast_flood_risk, dist], layout = cuxfilter.layouts.feature_and_base, theme = cuxfilter.themes.rapids) 


### Display the visualization

In [ ]:
d.show()
d.app(sidebar_width = 250)